In [111]:
workspace()
using Bolas
using Derivatives


    +(Bola,Any) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:185
is ambiguous with: 
    +(Any,Bola) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:184.
To fix, define 
    +(Bola,Bola)
before the new definition.
    +(Bola,Any) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:185
is ambiguous with: 
    +(Any,Bola) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:184.
To fix, define 
    +(Bola,Bola)
before the new definition.
    +(Bola,Any) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:185
is ambiguous with: 
    +(Any,Bola) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:184.
To fix, define 
    +(Bola,Bola)
before the new definition.
    +(Bola,Any) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:185
is ambiguous with: 
    +(Any,Bola) at /home/pablo1/Documents/Tareas/ProyectoFinalMNA/Bolas.jl:190.
To fix, define 
    +(Bola,Bola)
before the new definition.
    +(Bola,Any) at /home/pablo1/Documents/Ta

In [115]:
A=Bola(0,1)
B=Bola(1.2,1)

interseccion_de_bolas(B,A)

Bola(0.6,0.4)

In [116]:
function quitar_no_deseadas(F::Function,B,x)
    
    temp=Bola[]
    dF(x)=F(makex(x))
    
    for i=1:length(B)
        if contiene(dF(B[i]).d,x) == false
            push!(temp,B[i])
        end
    end
    
    return temp
    
    
end

    

quitar_no_deseadas (generic function with 1 method)

In [117]:
f(x)=exp(x)-1
quitar_no_deseadas(f,[A,B],10)

2-element Array{Bola,1}:
 Bola(0,1)  
 Bola(1.2,1)

In [84]:
bisectar_bolas([A,B])

4-element Array{Bola,1}:
 Bola(-0.5,0.5)
 Bola(0.5,0.5) 
 Bola(0.7,0.5) 
 Bola(1.7,0.5) 

In [118]:
function operador_de_newton_bola(F::Function,B::Bola)
    
    dF(x)=F(makex(x))
    m=Bola(B.centro,0)
    
    return (m-F(m)/(dF(B).d))
    
end

operador_de_newton_bola (generic function with 1 method)

In [110]:
f(B)

Bola(2.3201169227365472,3.3201169227365472)

In [51]:
contiene(dF(B).d,0)

false

In [119]:
function metodo_newton_bola(F::Function, B::Bola)
    
    X=[B]
    loop=0
    
    while loop < 50
        
        X=bisectar_bolas(X)
        X=quitar_no_deseadas(F,X,0)
        
        for i=1:length(X)
            
            @show X[i]=operador_de_newton_bola(F,X[i])
        end
        
     loop +=1
        
    end
    
    return(X)
    
end

metodo_newton_bola (generic function with 1 method)

In [120]:
metodo_newton_bola(f,Bola(2,1))

X[i] = operador_de_newton_bola(F,X[i]) => Bola(0.7231301601484298,7.8019239264248)
X[i] = operador_de_newton_bola(F,X[i]) => Bola(1.5820849986238987,68.11533257093657)


0-element Array{Bola,1}

In [77]:
interseccion_de_bolas(Bola(0.7999999999999999,0.6),Bola(1.7999999999999998,0.3999999999999999))

Bola(1.4,0.0)

In [102]:
f(1.2)

1.24